In [31]:
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import datetime

# 1. API 불러오기

- 기상청 단기예보 조회 서비스(https://www.data.go.kr/tcs/dss/selectApiDataDetailView.do?publicDataPk=15084084)

In [38]:
# 현재 날짜, 현재 시각
now = datetime.datetime.now()
base_date = now.date().strftime("%Y%m%d")

# 발표 시각을 현재시간 1시간 전으로 조회 (예측시간을 현재시간에 맞추기 위함)
base_time = (now - datetime.timedelta(hours=1)).strftime("%H%M")

In [39]:
key = 'POt4cCBFEG6ttsuSzsLdyQDtxAjMUVHZvy40cGESFGdxUv7ggP7qGYhXb8lpgM9epQZi8RZ7ZFgxWBDvulRAzA=='
url = 'http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getUltraSrtFcst'

x = 55
y = 127


params = {'ServiceKey':key,
          'numOfRows':1000,
          'pageNo':1,
          'base_date':base_date,
          'base_time':base_time,
         'nx':x,
         'ny':y}

In [40]:
res = requests.get(url, params=params)
res

<Response [200]>

In [41]:
soup = BeautifulSoup(res.text,'xml')
soup

<?xml version="1.0" encoding="utf-8"?>
<response><header><resultCode>00</resultCode><resultMsg>NORMAL_SERVICE</resultMsg></header><body><dataType>XML</dataType><items><item><baseDate>20230322</baseDate><baseTime>0930</baseTime><category>LGT</category><fcstDate>20230322</fcstDate><fcstTime>1000</fcstTime><fcstValue>0</fcstValue><nx>55</nx><ny>127</ny></item><item><baseDate>20230322</baseDate><baseTime>0930</baseTime><category>LGT</category><fcstDate>20230322</fcstDate><fcstTime>1100</fcstTime><fcstValue>0</fcstValue><nx>55</nx><ny>127</ny></item><item><baseDate>20230322</baseDate><baseTime>0930</baseTime><category>LGT</category><fcstDate>20230322</fcstDate><fcstTime>1200</fcstTime><fcstValue>0</fcstValue><nx>55</nx><ny>127</ny></item><item><baseDate>20230322</baseDate><baseTime>0930</baseTime><category>LGT</category><fcstDate>20230322</fcstDate><fcstTime>1300</fcstTime><fcstValue>0</fcstValue><nx>55</nx><ny>127</ny></item><item><baseDate>20230322</baseDate><baseTime>0930</baseTime><cate

In [43]:
items = soup.findAll('item')
items

[<item><baseDate>20230322</baseDate><baseTime>0930</baseTime><category>LGT</category><fcstDate>20230322</fcstDate><fcstTime>1000</fcstTime><fcstValue>0</fcstValue><nx>55</nx><ny>127</ny></item>,
 <item><baseDate>20230322</baseDate><baseTime>0930</baseTime><category>LGT</category><fcstDate>20230322</fcstDate><fcstTime>1100</fcstTime><fcstValue>0</fcstValue><nx>55</nx><ny>127</ny></item>,
 <item><baseDate>20230322</baseDate><baseTime>0930</baseTime><category>LGT</category><fcstDate>20230322</fcstDate><fcstTime>1200</fcstTime><fcstValue>0</fcstValue><nx>55</nx><ny>127</ny></item>,
 <item><baseDate>20230322</baseDate><baseTime>0930</baseTime><category>LGT</category><fcstDate>20230322</fcstDate><fcstTime>1300</fcstTime><fcstValue>0</fcstValue><nx>55</nx><ny>127</ny></item>,
 <item><baseDate>20230322</baseDate><baseTime>0930</baseTime><category>LGT</category><fcstDate>20230322</fcstDate><fcstTime>1400</fcstTime><fcstValue>0</fcstValue><nx>55</nx><ny>127</ny></item>,
 <item><baseDate>20230322

In [44]:
cols = ['baseDate','baseTime','category','fcstDate','fcstTime','fcstValue','nx','ny']

### api 요소 데이터프레임으로 정리하기

In [45]:
dict_list = []
for item in items:
    dict = {}    
    for col in cols:
        dict[col] = item.find(col).text
    dict_list.append(dict)

In [46]:
dict_list

[{'baseDate': '20230322',
  'baseTime': '0930',
  'category': 'LGT',
  'fcstDate': '20230322',
  'fcstTime': '1000',
  'fcstValue': '0',
  'nx': '55',
  'ny': '127'},
 {'baseDate': '20230322',
  'baseTime': '0930',
  'category': 'LGT',
  'fcstDate': '20230322',
  'fcstTime': '1100',
  'fcstValue': '0',
  'nx': '55',
  'ny': '127'},
 {'baseDate': '20230322',
  'baseTime': '0930',
  'category': 'LGT',
  'fcstDate': '20230322',
  'fcstTime': '1200',
  'fcstValue': '0',
  'nx': '55',
  'ny': '127'},
 {'baseDate': '20230322',
  'baseTime': '0930',
  'category': 'LGT',
  'fcstDate': '20230322',
  'fcstTime': '1300',
  'fcstValue': '0',
  'nx': '55',
  'ny': '127'},
 {'baseDate': '20230322',
  'baseTime': '0930',
  'category': 'LGT',
  'fcstDate': '20230322',
  'fcstTime': '1400',
  'fcstValue': '0',
  'nx': '55',
  'ny': '127'},
 {'baseDate': '20230322',
  'baseTime': '0930',
  'category': 'LGT',
  'fcstDate': '20230322',
  'fcstTime': '1500',
  'fcstValue': '0',
  'nx': '55',
  'ny': '127'}

#### 필요한 요소만 사용
- SKY(하늘상태코드) : 맑음(1), 구름많음(3), 흐림(4) 
- PTY(강수형태코드) : 없음(0), 비(1), 비/눈(2), 눈(3), 빗방울(5), 빗방울눈날림(6), 눈날림(7)

In [47]:
df = pd.DataFrame(dict_list, columns = cols)
sky = df[df['category']=='SKY'].iloc[0].fcstValue
sky

'4'

In [48]:
pty = df[df['category']=='PTY'].iloc[0].fcstValue
pty

'0'

### 구별 기상청 제공 격자 위경도 추출

In [49]:
xy_df = pd.read_excel('기상청41_단기예보 조회서비스_오픈API활용가이드_격자_위경도(20221027).xlsx')
xy_df.head(5)

,구분,행정구역코드,1단계,2단계,3단계,격자 X,격자 Y,경도(시),경도(분),경도(초),위도(시),위도(분),위도(초),경도(초/100),위도(초/100),위치업데이트
0,kor,1100000000,서울특별시,NaN,NaN,60,127,126,58,48.03,37,33,48.85,126.980008,37.563569,NaN
1,kor,1111000000,서울특별시,종로구,NaN,60,127,126,58,53.91,37,34,13.36,126.981642,37.570378,NaN
2,kor,1111051500,서울특별시,종로구,청운효자동,60,127,126,58,14.35,37,35,2.89,126.970652,37.584137,NaN
3,kor,1111053000,서울특별시,종로구,사직동,60,127,126,58,15.44,37,34,23.77,126.970956,37.573269,NaN
4,kor,1111054000,서울특별시,종로구,삼청동,60,127,126,59,2.32,37,34,56.73,126.983978,37.582425,NaN


In [50]:
# 서울시만 추출
xy_seoul = xy_df[xy_df['1단계'] == '서울특별시']
xy_seoul

,구분,행정구역코드,1단계,2단계,3단계,격자 X,격자 Y,경도(시),경도(분),경도(초),위도(시),위도(분),위도(초),경도(초/100),위도(초/100),위치업데이트
0,kor,1100000000,서울특별시,NaN,NaN,60,127,126,58,48.03,37,33,48.85,126.980008,37.563569,NaN
1,kor,1111000000,서울특별시,종로구,NaN,60,127,126,58,53.91,37,34,13.36,126.981642,37.570378,NaN
2,kor,1111051500,서울특별시,종로구,청운효자동,60,127,126,58,14.35,37,35,2.89,126.970652,37.584137,NaN
3,kor,1111053000,서울특별시,종로구,사직동,60,127,126,58,15.44,37,34,23.77,126.970956,37.573269,NaN
4,kor,1111054000,서울특별시,종로구,삼청동,60,127,126,59,2.32,37,34,56.73,126.983978,37.582425,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447,kor,1174065000,서울특별시,강동구,성내제2동,62,126,127,7,53.71,37,31,46.57,127.131586,37.529603,NaN
448,kor,1174066000,서울특별시,강동구,성내제3동,63,126,127,8,5.59,37,31,24.14,127.134886,37.523372,NaN
449,kor,1174068500,서울특별시,강동구,길동,63,126,0,0,0.00,0,0,0.00,127.146168,37.539217,20150908.0
450,kor,1174069000,서울특별시,강동구,둔촌제1동,63,126,127,8,19.79,37,31,14.70,127.138831,37.520750,NaN


In [51]:
# 구 이름 확인
gu = xy_seoul['2단계'].unique()[1:]
gu

array(['종로구', '중구', '용산구', '성동구', '광진구', '동대문구', '중랑구', '성북구', '강북구',
       '도봉구', '노원구', '은평구', '서대문구', '마포구', '양천구', '강서구', '구로구', '금천구',
       '영등포구', '동작구', '관악구', '서초구', '강남구', '송파구', '강동구'], dtype=object)

In [52]:
gu[0]

'종로구'

In [53]:
xy_seoul[xy_seoul['2단계'] == gu[1]].iloc[0]['격자 Y']

127

#### 구별 x,y 데이터프레임 생성

In [54]:
xy = pd.DataFrame([], columns=['구', 'x', 'y'])

for i in range(len(gu)):
    xy.loc[i] = [xy_seoul[xy_seoul['2단계'] == gu[i]].iloc[0]['2단계'],
                  xy_seoul[xy_seoul['2단계'] == gu[i]].iloc[0]['격자 X'],
                  xy_seoul[xy_seoul['2단계'] == gu[i]].iloc[0]['격자 Y']]
xy

,구,x,y
0,종로구,60,127
1,중구,60,127
2,용산구,60,126
3,성동구,61,127
4,광진구,62,126
5,동대문구,61,127
6,중랑구,62,128
7,성북구,61,127
8,강북구,61,128
9,도봉구,61,129


In [225]:
xy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25 entries, 0 to 24
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   구       25 non-null     object
 1   x       25 non-null     int64 
 2   y       25 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 800.0+ bytes


#### csv 파일로 내보내기

In [11]:
xy.to_csv('./기상청제공위경도.csv')

### 구 입력해서 현재날씨 조회해보기

In [55]:
input_gu = input('구를 입력하세요')

구를 입력하세요강남구


In [56]:
x = xy[xy['구']==input_gu].x
y = xy[xy['구']==input_gu].y

In [57]:
key = 'POt4cCBFEG6ttsuSzsLdyQDtxAjMUVHZvy40cGESFGdxUv7ggP7qGYhXb8lpgM9epQZi8RZ7ZFgxWBDvulRAzA=='
url = 'http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getUltraSrtFcst'

now = datetime.datetime.now()
base_date = now.date().strftime("%Y%m%d")
base_time = now.time().strftime("%H%M")

params = {'ServiceKey':key,
          'numOfRows':1000,
          'pageNo':1,
          'base_date':base_date,
          'base_time':base_time,
         'nx':x,
         'ny':y}

In [58]:
print(x, y , base_date, base_time)

22    61
Name: x, dtype: int64 22    126
Name: y, dtype: int64 20230322 1019


In [59]:
res = requests.get(url, params=params)
soup = BeautifulSoup(res.text,'xml')
items = soup.findAll('item')

In [60]:
cols = ['baseDate','baseTime','category','fcstDate','fcstTime','fcstValue','nx','ny']
dict_list = []
for item in items:
    dict = {}    
    for col in cols:
        dict[col] = item.find(col).text
    dict_list.append(dict)

In [68]:
df = pd.DataFrame(dict_list, columns = cols)
sky = df[df['category']=='SKY'].iloc[0].fcstValue
pty = df[df['category']=='PTY'].iloc[0].fcstValue
print(sky,pty)

4 0


#### sky, pyt 코드를 이용해 예보 날씨를 맑음/흐림/눈/비로 구분하기

In [69]:
if (pty == '1') | (pty == '5'):
    print(f'{input_gu} 현재 날씨 : 비')
elif (pty == '3') | (pty == '7'):
    print(f'{input_gu} 현재 날씨 : 눈')
elif (sky == '1') & (pty == '0'):
    print(f'{input_gu} 현재 날씨 :맑음')
elif ((sky == '3') & (pty == '0')) | ((sky == '4') & (pty == '0')) :
    print(f'{input_gu} 현재 날씨 : 흐림')
else:
    print(f'{input_gu} 현재 날씨 : 안좋음')

강남구 현재 날씨 : 흐림


In [70]:
#### 발표 시각을 현재시간 1시간 전으로 조회 (예측시간을 현재시간에 맞추기 위함)
base_time = (now - datetime.timedelta(hours=1)).strftime("%H%M")
base_time

'0919'

In [71]:
x = xy[xy['구']=='은평구'].x
y = xy[xy['구']=='은평구'].y
now = datetime.datetime.now()
base_date = now.date().strftime("%Y%m%d")
base_time = (now - datetime.timedelta(hours=1)).strftime("%H%M")
params = {'ServiceKey':key,
          'numOfRows':1000,
          'pageNo':1,
          'base_date':base_date,
          'base_time':base_time,
          'nx':x,
          'ny':y}
    
res = requests.get(url, params=params)
soup = BeautifulSoup(res.text,'xml')
items = soup.findAll('item')
    
cols = ['baseDate','baseTime','category','fcstDate','fcstTime','fcstValue','nx','ny']
dict_list = []
for item in items:
    dict = {}    
    for col in cols:
        dict[col] = item.find(col).text
    dict_list.append(dict)
    
df = pd.DataFrame(dict_list, columns = cols)

In [72]:
df[df['category']=='SKY']

,baseDate,baseTime,category,fcstDate,fcstTime,fcstValue,nx,ny
18,20230322,0930,SKY,20230322,1000,4,59,127
19,20230322,0930,SKY,20230322,1100,4,59,127
20,20230322,0930,SKY,20230322,1200,4,59,127
21,20230322,0930,SKY,20230322,1300,4,59,127
22,20230322,0930,SKY,20230322,1400,4,59,127
23,20230322,0930,SKY,20230322,1500,4,59,127


In [73]:
df[df['category']=='PTY']

,baseDate,baseTime,category,fcstDate,fcstTime,fcstValue,nx,ny
6,20230322,0930,PTY,20230322,1000,0,59,127
7,20230322,0930,PTY,20230322,1100,0,59,127
8,20230322,0930,PTY,20230322,1200,0,59,127
9,20230322,0930,PTY,20230322,1300,0,59,127
10,20230322,0930,PTY,20230322,1400,0,59,127
11,20230322,0930,PTY,20230322,1500,0,59,127


### 현재 날씨 조회 함수로 만들어서 적용해보기

In [2]:
xy = pd.read_csv('./기상청제공위경도.csv')

In [74]:
def weather_now(gu):
    key = 'POt4cCBFEG6ttsuSzsLdyQDtxAjMUVHZvy40cGESFGdxUv7ggP7qGYhXb8lpgM9epQZi8RZ7ZFgxWBDvulRAzA=='
    url = 'http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getUltraSrtFcst'
    
    x = xy[xy['구']==gu].x
    y = xy[xy['구']==gu].y
    now = datetime.datetime.now()
    base_date = now.date().strftime("%Y%m%d")
    base_time = (now - datetime.timedelta(hours=1)).strftime("%H%M")

    params = {'ServiceKey':key,
              'numOfRows':1000,
              'pageNo':1,
              'base_date':base_date,
              'base_time':base_time,
             'nx':x,
             'ny':y}
    
    res = requests.get(url, params=params)
    soup = BeautifulSoup(res.text,'xml')
    items = soup.findAll('item')
    
    cols = ['baseDate','baseTime','category','fcstDate','fcstTime','fcstValue','nx','ny']
    dict_list = []
    for item in items:
        dict = {}    
        for col in cols:
            dict[col] = item.find(col).text
        dict_list.append(dict)
    
    df = pd.DataFrame(dict_list, columns = cols)
    sky = df[df['category']=='SKY'].iloc[0].fcstValue
    pty = df[df['category']=='PTY'].iloc[0].fcstValue
    
    if (pty == '1') | (pty == '5'):
        weather = ['비','']
    elif (pty == '3') | (pty == '7'):
        weather = ['눈', '']
    elif (pty == '2') | (pty == '6'):
        weather = ['눈','비']
    elif (sky == '1') & (pty == '0'):
        weather = ['맑음', '']
    elif ((sky == '3') & (pty == '0')) | ((sky == '4') & (pty == '0')) :
        weather = ['흐림','']
    
    return weather

In [75]:
weather_now('은평구')

['흐림', '']

In [76]:
weather_now('강남구')

['흐림', '']

# 2. 사고건수 데이터와 연결하기

In [4]:
weather_data = pd.read_csv('./기상상태별 교통사고 전처리 데이터.csv', encoding='cp949')
weather_data

,Unnamed: 0,지자체,기상상태,발생건수
0,0,강동구,맑음,21
1,1,동대문구,맑음,29
2,2,동작구,맑음,33
3,3,마포구,맑음,66
4,4,서초구,맑음,70
...,...,...,...,...
120,120,서대문구,눈,0
121,121,송파구,눈,0
122,122,양천구,눈,0
123,123,중구,눈,0


In [5]:
total_acd = weather_data.발생건수.sum()
total_acd

1016

In [6]:
weather_data.columns

Index(['Unnamed: 0', '지자체', '기상상태', '발생건수'], dtype='object')

In [7]:
weather_data.iloc[2]['지자체']

'동작구'

#### 구별 날씨별 사고 건수 조회해보기

In [77]:
gu = '강남구'
weather = ['눈','비']
cnt = []

for w in weather:
    for i in range(len(weather_data)):
        if (weather_data.iloc[i]['지자체'] == gu) & (weather_data.iloc[i]['기상상태'] == w):
            cnt.append(weather_data.iloc[i]['발생건수'])
        else: continue 
    
cnt

[1, 7]

#### 사고율 계산

In [78]:
total_gu = weather_data[weather_data['지자체']==gu].발생건수.sum()
acd_rate = cnt / total_gu *100

In [79]:
w_dic = {}

if weather[1] == '':
    w_dic[weather[0]] = round(acd_rate[0],2)
else:
    w_dic[weather[0]] = round(acd_rate[0],2)
    w_dic[weather[1]] = round(acd_rate[1],2)
print(w_dic)

{'눈': 0.47, '비': 3.27}


In [80]:
w_dic['눈']

0.47

### 구별 날씨별 날씨 및 사고율 출력 함수 정의

In [81]:
def weather_acd(gu):
    weather = weather_now(gu)
    cnt=[]
    for w in weather:
        cnt1 = 0
        for i in range(len(weather_data)):
            if (weather_data.iloc[i]['지자체'] == gu) & (weather_data.iloc[i]['기상상태'] == w):
                cnt.append(weather_data.iloc[i]['발생건수']) 
            else: continue
    
    # 구별 사고건수 총합
    total_gu = weather_data[weather_data['지자체']==gu].발생건수.sum()
    # 사고율
    acd_rate = cnt/total_gu *100
    
    w_dic = {}
    if weather[1] == '':
        w_dic[weather[0]] = round(acd_rate[0],2)
    else:
        w_dic[weather[0]] = round(acd_rate[0],2)
        w_dic[weather[1]] = round(acd_rate[1],2)
    
    return w_dic

In [82]:
weather_acd('강남구')

{'흐림': 3.74}